## [SerpstatDomainProcedure.getDomainsInfo Method](https://serpstat.com/api/412-summarnij-otchet-po-domenu-v4-serpstatdomainproceduregetdomainsinfo/)

In [2]:
from datetime import datetime
import pandas as pd
import requests
from pandas import json_normalize
import time

### Key Features

* Provides Key Domain Metrics: Returns essential information about a domain, including SEO metrics such as organic traffic, backlinks, and rankings.

* Supports SEO Audits: Helps in performing domain audits by providing data that reflects overall domain health and performance.

* Aids Competitive Benchmarking: Allows you to compare your domain’s performance against competitors in terms of visibility, keyword rankings, and backlink profiles.


| Column               | Description                                                                                             |
|----------------------|---------------------------------------------------------------------------------------------------------|
| domain               | The domain for which the data is being retrieved.                                                      |
| visible              | The visibility score of the domain, indicating how much of its traffic is coming from organic search.    |
| keywords             | Total number of keywords the domain ranks for.                                                          |
| traff                | The total traffic or visits the domain receives.                                                        |
| visible_dynamic      | The dynamic visibility score, indicating changes in the domain's organic search visibility over time.    |
| keywords_dynamic     | The number of keywords the domain is dynamically ranking for over time.                                 |
| traff_dynamic        | The fluctuation in the domain's traffic over time.                                                      |
| ads_dynamic          | The change in the number of ads or paid campaigns the domain is running.                                |
| new_keywords         | Number of new keywords the domain started ranking for during the specified period.                      |
| out_keywords         | Number of keywords the domain no longer ranks for during the specified period.                          |
| rised_keywords       | Number of keywords where the domain saw an improvement in ranking.                                      |
| down_keywords        | Number of keywords where the domain saw a decline in ranking.                                           |
| ad_keywords          | The number of keywords associated with paid ads during the specified period.                            |
| ads                  | The total number of paid ads the domain is running.                                                     |
| prev_date            | The previous date for which data was retrieved, used for comparison with the current data snapshot.      |


In [ ]:
path = r"C:\Users\nickj\OneDrive\Desktop\Xoinx\Xoinx Value Prospecting Firms\develomark_serp_data_12.16.24.xlsx"

riseoak = pd.read_excel(path, sheet_name='targets')

lst_of_domains = riseoak['Target Domains'].drop_duplicates().to_list()

lst_of_domains = lst_of_domains[1:]
len(lst_of_domains)

In [ ]:
# API setup
api_token = '185b0a40fdd3abccb33f353c8e3e2918'
api_url_pattern = 'https://api.serpstat.com/v{version}?token={token}'
api_url = api_url_pattern.format(version=4, token=api_token)

# Retry parameters
MAX_RETRIES = 5
RETRY_DELAY = 5
BATCH_SIZE = 10  # Adjust this size as needed, 10 domains per request

# Create a session for persistent connection
session = requests.Session()

# Function to split the list into batches
def split_into_batches(lst, batch_size):
    for i in range(0, len(lst), batch_size):
        yield lst[i:i + batch_size]

# Function to make requests with retry logic
def make_request(session, url, data, retries=MAX_RETRIES):
    for attempt in range(retries):
        try:
            resp = session.post(url, json=data)
            
            if resp.status_code == 200:
                return resp.json()
            elif resp.status_code == 504:
                print(f"504 Gateway Timeout. Retrying in {RETRY_DELAY} seconds...")
                time.sleep(RETRY_DELAY)
            else:
                resp.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f"Error: {e}. Retrying in {RETRY_DELAY} seconds...")
            time.sleep(RETRY_DELAY)
    raise Exception(f"Failed after {retries} retries")

def process_domains_in_batches(lst_of_domains):
    all_data = []

    # Process the list of domains in batches
    for domain_batch in split_into_batches(lst_of_domains, BATCH_SIZE):
        data = {
            "id": "1",
            "method": "SerpstatDomainProcedure.getDomainsInfo",
            "params": {
                "domains": domain_batch,
                "se": "g_us",  # Google US
                "filters": {}
            }
        }

        try:
            # Make the request for each batch
            result = make_request(session, api_url, data)
            all_data.extend(result['result']['data'])  # Collect results from each batch
            
            print(f"Count of {len(all_data)}")
            time.sleep(3)

        except Exception as e:
            print(f"Error processing batch {domain_batch}: {e}")

    return all_data

try:
    # Process the domains in batches
    tabular_data = process_domains_in_batches(lst_of_domains)
    
    # Convert the collected data to a DataFrame
    df = pd.json_normalize(tabular_data)

except Exception as e:
    print(f"An error occurred: {e}")

df.to_clipboard()

In [ ]:
df.to_clipboard()
df

## Column Explanations
#### domain
* Meaning: The name of the website or domain being analyzed.
* Example: rulonco.com, prosites.com.

#### visible
* Meaning: This represents the visibility score of the domain, often indicating the proportion of its pages that are appearing in search engine results relative to other domains.
* Example: 0.0167 for rulonco.com indicates a low visibility score.

#### keywords
* Meaning: The total number of keywords for which the domain ranks in search engine results.
* Example: 6421 for rulonco.com means it ranks for 6,421 different keywords.

#### traff
* Meaning: The estimated traffic to the domain, typically measured in visits or sessions per month.
* Example: 7299 for rulonco.com indicates it receives approximately 7,299 visits per month.

#### visible_dynamic
* Meaning: This shows the dynamic visibility score, which may reflect changes in visibility over a specific period (e.g., the last month).
* Example: 0.0002 for rulonco.com, indicating a slight increase or stability in visibility.

#### keywords_dynamic
* Meaning: This shows the change in the number of keywords for which the domain ranks, reflecting growth or decline.
* Example: 177 for rulonco.com means an increase of 177 keywords since the previous measurement.

#### traff_dynamic
* Meaning: The change in estimated traffic compared to the previous measurement, indicating whether traffic has increased or decreased.
* Example: 7299 for rulonco.com suggests a positive change in traffic.

#### ads_dynamic
* Meaning: This represents changes in the number of ads associated with the domain, which may indicate marketing activity.
* Example: 0 for rulonco.com suggests no change in the number of ads.

#### new_keywords
* Meaning: The number of new keywords that the domain has started ranking for since the last measurement.
* Example: 669 for rulonco.com indicates the domain has begun ranking for 669 new keywords.

#### out_keywords
* Meaning: The number of keywords that the domain has dropped from its rankings since the last measurement.
* Example: 492 for rulonco.com means it has lost ranking for 492 keywords.

#### rised_keywords
* Meaning: The number of keywords that have improved in ranking since the previous measurement.
* Example: 278 for rulonco.com indicates that 278 keywords have risen in ranking.

#### down_keywords
* Meaning: The number of keywords that have declined in ranking.
* Example: 397 for rulonco.com shows that 397 keywords have dropped in ranking.

#### ad_keywords
* Meaning: The number of keywords for which ads are running for the domain.
* Example: 0 for rulonco.com indicates no keywords are currently being targeted with ads.

#### ads
* Meaning: The total number of ads being run for the domain.
* Example: 0 for rulonco.com suggests that there are no active ads.

#### prev_date
* Meaning: The date of the previous measurement, providing a timestamp for the data.
* Example: 2024-09-06 indicates when the last analysis was performed.